# Detection of modified images or videos using Neural Networks

## Importing the libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageChops, ImageFilter
import pandas as pd
import os
import io
import shutil
from kaggle.api.kaggle_api_extended import KaggleApi

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.utils import to_categorical
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Input, Lambda, Resizing, GlobalAveragePooling2D
from keras.applications import ResNet50, MobileNet, VGG16
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix
import itertools

K.clear_session()
tf.compat.v1.reset_default_graph()
tf.compat.v1.enable_eager_execution()

## Downloading the dataset

In [ ]:
URL = "sophatvathana/casia-dataset"
PATH_DATASET = './../dataset/'

def download_dataset():
    api = KaggleApi()
    api.authenticate()
    print("Downloading files...")
    api.dataset_download_files('sophatvathana/casia-dataset', path=PATH_DATASET, unzip=True)

    print("\rDownload complete.")


def clean_directory():
    print("Moving folder...")
    os.rename(PATH_DATASET+"CASIA2/Au", PATH_DATASET+"Au")
    os.rename(PATH_DATASET+"CASIA2/Tp", PATH_DATASET+"Tp")
    
    print("Cleaning directory...")
    shutil.rmtree(PATH_DATASET+"casia")
    shutil.rmtree(PATH_DATASET+"CASIA1")
    shutil.rmtree(PATH_DATASET+"CASIA2")
    os.remove(PATH_DATASET+"Tp/Thumbs.db")
    os.remove(PATH_DATASET+"Au/Thumbs.db")
    print("Cleaning complete.")

In [ ]:
if not os.path.exists(PATH_DATASET+"Au"):
    download_dataset()
    clean_directory()
else:
    print("Dataset already Downloaded.")

In [ ]:
REAL_IMAGE_PATH = '../dataset/Au'
FAKE_IMAGE_PATH = "../dataset/Tp"
IMG_SIZE = (256, 256)
CLASS = ['Manipulated', 'Original']

In [ ]:
cabezera = "category", "image", "real"
df_au = pd.DataFrame(columns=cabezera)

for idx, file in enumerate(os.listdir(REAL_IMAGE_PATH)):
    img = cv2.imread(os.path.join(REAL_IMAGE_PATH, file))
    img = cv2.resize(img, IMG_SIZE)
    #img_np = np.array(img)
    category = file.split("_")

    df_au = pd.concat([df_au, pd.DataFrame([[category[1], img, 1]], columns=cabezera)], ignore_index=True)

df_au.head()

In [ ]:
#remove categoriy txt because it is not in the dataset
df_au = df_au[df_au.category != "txt"]
df_au = df_au[df_au.category != "ind"]
#mezclar el dataframe
df_au = df_au.sample(frac=1).reset_index(drop=True)

In [ ]:
#mezclar el dataframe
df_au = df_au.sample(frac=1).reset_index(drop=True)

Realizaremos el mismo proceso con las imagenes modificadas

In [ ]:
cabezera = "category", "image", "region", "real"
key_list = ["ani", "arc", "art", "cha", "nat", "pla", "sec"]
df_tp = pd.DataFrame(columns=cabezera)

for file in os.listdir(FAKE_IMAGE_PATH):
    #convert image to np array
    img = cv2.imread(os.path.join(FAKE_IMAGE_PATH, file))
    img = cv2.resize(img, IMG_SIZE)
    #img_np = np.array(img)
    category = file.split("_")
    category[5] = category[5][:3]
    df_tp = pd.concat([df_tp, pd.DataFrame([[category[5], img, category[1], 0]], columns=cabezera)], ignore_index=True)

df_tp.head()

In [ ]:
df_tp = df_tp[df_tp.category != "txt"]
df_tp = df_tp[df_tp.category != "ind"]
df_tp = df_tp.sample(frac=1).reset_index(drop=True)

In [ ]:
df = pd.concat([df_au, df_tp], ignore_index=True)

In [ ]:
def plot_ela_images(original, images: list, qualities: list):
    fig = plt.figure(figsize=(20, 20))
    fig.add_subplot(1, 4, 1)
    plt.title("Original")
    plt.imshow(original)
    plt.axis('off')
    
    for i, image in enumerate(images):
        fig.add_subplot(1, 4, i+2)
        plt.title("Quality: " + str(qualities[i]))
        plt.imshow(image)
        plt.axis('off')
    plt.show()

def ela(orig_img, quality=90):
    _, buffer = cv2.imencode('.jpg', orig_img, [cv2.IMWRITE_JPEG_QUALITY, quality])
    compressed_img = cv2.imdecode(np.frombuffer(buffer, np.uint8), cv2.IMREAD_COLOR)

    diff = 15 * cv2.absdiff(orig_img, compressed_img)
    return diff

## Creating Model

### Creating and splitting the dataset

In [ ]:
# Preprocesar las imágenes
tensors_x = [tf.convert_to_tensor(ela(image)) for image in df['image']]
tensors_y = [tf.convert_to_tensor(label) for label in df['real']]

In [ ]:
X = np.array(tensors_x)
Y = np.array(tensors_y)

# Dividir el conjunto de datos en entrenamiento validacion y prueba
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

### Creating the checkpoints

In [ ]:
metrics = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR')
]

early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=5, 
    verbose=0, 
    mode='auto', 
    baseline=None, 
    restore_best_weights=False
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)


model_chekpoint = ModelCheckpoint(
    filepath='./../model/checkpoints', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True,
    save_weights_only=True, 
    mode='auto', 
    save_freq='epoch'
)

tensor_board = TensorBoard(
    log_dir='./../model/logs',
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    update_freq='epoch',
    profile_batch=2,
    embeddings_freq=0,
    embeddings_metadata=None
)

callbacks = [early_stopping, model_chekpoint, tensor_board]
optimizer = Adam(learning_rate=0.0001)

### Creating the model fron scratch

detect_manipulated_images_model_scratch_v1.h5 8min 31s 12 epocas bacth_size=32

loss: 1.6352 - accuracy: 0.7355 - precision: 0.9418 - recall: 0.3194 - auc: 0.8527 - prc: 0.8239

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), activation=None, input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(5, 5), activation=None, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(5, 5), activation=None, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation=None, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

### Creating the model using transfer learning (ResNet50)

detect_manipulated_images_model_resnet50_v1.h5 10m 46s 7 epocas bacth_size=16

loss: 0.4613 - accuracy: 0.8307 - precision: 0.8105 - recall: 0.7201 - auc: 0.9256 - prc: 0.8760



In [ ]:
inputs = Input(shape=(256, 256, 3))

model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

x = Flatten()(model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=model.inputs, outputs=predictions)

### Creating the model using transfer learning (MobileNet)

detect_manipulated_images_model_mobilenet_v1.h5 4min 37s 9 epocas bacth_size=32

loss: 0.6656 - accuracy: 0.8864 - precision: 0.8028 - recall: 0.9268 - auc: 0.9350 - prc: 0.8523

In [ ]:
inputs = Input(shape=(256, 256, 3))
model = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

x = GlobalAveragePooling2D()(model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=model.inputs, outputs=predictions)

detect_manipulated_images_model_mobilenet_v2.h5 1min 47s 12 epocas bacth_size=32

loss: 0.5404 - accuracy: 0.7285 - precision: 0.7123 - recall: 0.4715 - auc: 0.7986 - prc: 0.6622

In [ ]:
model = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Congelar todas las capas del modelo pre-entrenado para que no se modifiquen durante el entrenamiento
for layer in model.layers:
    layer.trainable = False

# Agregar capas adicionales en la parte superior del modelo pre-entrenado
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Definir el modelo completo
model = Model(inputs=model.inputs, outputs=predictions)

### Creating the model using transfer learning (VGG16)

detect_manipulated_images_model_vgg16_v1.h5 36m 11s 15 epocas bacth_size=32

loss: 0.3823 - accuracy: 0.8662 - precision: 0.8022 - recall: 0.8571 - auc: 0.9345 - prc: 0.8679

In [ ]:
# Cargar la arquitectura pre-entrenada VGG-16 sin las capas completamente conectadas
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Definir una nueva capa de salida personalizada
x = vgg16.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Construir el modelo final que incluye VGG-16 y la nueva capa de salida
model = Model(inputs=vgg16.inputs, outputs=predictions)

### Training the model

In [ ]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)
history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val), callbacks=callbacks)

### Evaluating the model

In [ ]:
model.evaluate(x_test, y_test)

### Showing the results

In [ ]:
def plot_metrics(history):
    metrics = ['accuracy', 'loss', 'prc', 'precision', 'recall']
    fig, axes = plt.subplots(len(metrics), 1, figsize=(10, 10))
    
    for i, metric in enumerate(metrics):
        axes[i].plot(history.history[metric], label='train')
        axes[i].plot(history.history[f'val_{metric}'], label='val')
        axes[i].set_title(metric)
        axes[i].legend()
    
    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(model, X, y_true):
    y_pred = model.predict(X) > 0.5
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,6))
    plt.imshow(cm, cmap=plt.cm.Reds)
    plt.title('Confusion Matrix', fontsize=16)
    plt.ylabel('True label', fontsize=14)
    plt.xlabel('Predicted label', fontsize=14)
    plt.xticks([0, 1], ['Manipulated', 'Original'], fontsize=12)
    plt.yticks([0, 1], ['Manipulated', 'Original'], fontsize=12)
    plt.colorbar()
    for i in range(2):
        for j in range(2):
            plt.text(j, i, str(cm[i][j]), ha='center', va='center', fontsize=20)
    plt.show()

In [ ]:
plot_metrics(history)

In [ ]:
plot_confusion_matrix(model, x_test, y_test)

In [ ]:
model.save('./../model/detect_manipulated_images_model_resNet50.h5')

In [ ]:
tf.compat.v1.disable_eager_execution()
model = load_model('./../model/ela_models/detect_manipulated_images_model_mobileNet_v3.h5')

### Probando modelo con mapa de calor

In [ ]:
def get_last_conv_layer(model):
    for layer in reversed(model.layers):
        # Comprobar si la capa es una capa convolucional
        if isinstance(layer, Conv2D):
            return layer
    
    return None

def predict_with_heatmap(model, img_original):
    # Load image and convert to RGB
    img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
    img_original = cv2.resize(img_original, (256, 256))
    img = ela(img_original)
    img = np.expand_dims(img, axis=0)
    
    # Get predictions and last convolutional layer output
    preds = model.predict(img)
    last_conv_layer = get_last_conv_layer(model)
    last_conv_layer_model = Model(model.inputs, last_conv_layer.output)
    last_conv_output = last_conv_layer_model.predict(img)
    
    # Get class activation map
    class_idx = np.argmax(preds[0])
    class_output = model.output[:, class_idx]
    grads = K.gradients(class_output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.inputs], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, last_conv_output_value = iterate([img])
    for i in range(last_conv_output_value.shape[-1]):
        last_conv_output_value[:, :, i] *= pooled_grads_value[i]
    heatmap = np.mean(last_conv_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    
    # Resize heatmap to match original image size
    heatmap = cv2.resize(heatmap, (img.shape[2], img.shape[1]))
    
    # Convert heatmap to RGB format
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    # Overlay heatmap on original image
    superimposed_img = cv2.addWeighted(img_original, 0.6, heatmap, 0.4, 0)
    
    return img_original, superimposed_img, class_idx

def plot_heatmap(img_original, superimposed_img, class_idx):
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.title('Original image')
    plt.imshow(img_original)
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.title('Heatmap')
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()
    
    print('Predicted class:', class_idx)
    print('Predicted class name:', CLASS[class_idx])

In [ ]:
tf.compat.v1.disable_eager_execution()
model = load_model('./../model/ela_models/detect_manipulated_images_model_scratch_v1.h5')
path = '../dataset/Tp/Tp_D_CRD_S_O_ani10103_ani10111_10637.jpg'
img = cv2.imread(path)

original_img, heatmap_img, result = predict_with_heatmap(model, img)
plot_heatmap(original_img, heatmap_img, result)